In [86]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from config import get_tickers
from data_downloader import get_market_data
from portfolios_toolkit import markowitz_weights

### Building a Portfolio ###

In [87]:
stocks_tickers = get_tickers("6.1a")

stocks_tickers

In [88]:
sectors_tickers = get_tickers("6.1b")

sectors_tickers

In [89]:
# DataFrame to store everything
stock_returns_df = pd.DataFrame()

for ticker in stocks_tickers:
    df_stock = get_market_data(
        ticker=ticker, 
        start_date='2020-01-01', 
        end_date='2026-01-01', 
        returns=True
    )
    
    returns = df_stock['returns'].rename(ticker)
    
    stock_returns_df = pd.concat([stock_returns_df, returns], axis=1)

In [90]:
stock_returns_df

In [91]:
# Portfolio's Expected Returns
expected_returns = stock_returns_df.mean()
expected_returns.name = 'mean_returns'

expected_returns

In [92]:
# Covariance Matrix
covariance_matrix = stock_returns_df.cov()

covariance_matrix

In [93]:
# Correlation Matrix
correlation_matrix = stock_returns_df.corr()

correlation_matrix

In [94]:
# Obtain Weights
p_weights = markowitz_weights(
    expected_returns, 
    covariance_matrix,
    0.0010
)

In [95]:
# Create a Portfolio Weights DF
portfolio_weights = pd.Series(
    p_weights,
    index = stock_returns_df.columns,
    name = 'weights'
)

portfolio_weights

In [96]:
# Portfolio Returns
portfolio_returns = stock_returns_df @ p_weights
portfolio_returns.name = 'portfolio_returns'

portfolio_returns

In [97]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(stock_returns_df.cumsum(), label=stock_returns_df.columns, alpha=1)
plt.plot(portfolio_returns.cumsum(), label='Portfolio', color='black', alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show() 

### Getting a Benchmark: Using a Hypothetical Portfolio ###

In [98]:
# DataFrame to store everything
sector_returns_df = pd.DataFrame()

for ticker in sectors_tickers:
    df_stock = get_market_data(
        ticker=ticker, 
        start_date='2020-01-01', 
        end_date='2026-01-01', 
        returns=True
    )
    
    returns = df_stock['returns'].rename(ticker)
    
    sector_returns_df = pd.concat([sector_returns_df, returns], axis=1)

In [99]:
sector_returns_df

In [100]:
# Get the returns of each stock of our portfolio but in the IWY
bench_weights = pd.Series(
    [31.6, 9.6, 14.3, 10.6, 3.0, 8.7, 2.1],
    index = sector_returns_df.columns,
    name = 'weights'
)

bench_weights = bench_weights/100

bench_weights

In [101]:
# Normalized
norm_bench_weights = bench_weights/bench_weights.sum()

norm_bench_weights

In [102]:
# Build the Benchmark Returns
benchmark_returns = sector_returns_df @ norm_bench_weights
benchmark_returns.name = 'benchmark_returns'

benchmark_returns

In [103]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(benchmark_returns.cumsum(), label='Benchmark', alpha=1)
plt.plot(portfolio_returns.cumsum(), label='Portfolio', alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show() 

In [104]:
# Calculate the Mean Returns
portfolio_total_returns = portfolio_returns.cumsum().iloc[-1]
benchmark_total_returns = benchmark_returns.cumsum().iloc[-1]

print(portfolio_total_returns)
print(benchmark_total_returns)

In [105]:
# Calculate the Excess Returns
excess_returns = portfolio_total_returns - benchmark_total_returns

excess_returns

In [106]:
# Sectors List
sectors_list = [
    "Technology",
    "Health",
    "Financials",
    "Consumer",
    "Energy",
    "Industrials",
    "Real_Estate"
]

In [107]:
stock_returns_df.columns = sectors_list
sector_returns_df.columns = sectors_list

In [108]:
portfolio_weights.index = sectors_list
norm_bench_weights.index = sectors_list

The calculations of the effects in the Brinson-Fachler Model can be reviewed in the first section of this module's PDF. The mathematical derivations and explanations are provided there.

In [141]:
# Create a Comparison DF
comparison_df = pd.DataFrame(index = sectors_list)
comparison_df['portfolio_weights'] = portfolio_weights
comparison_df['benchmark_weights'] = norm_bench_weights
comparison_df['stocks_returns'] = stock_returns_df.cumsum().iloc[-1]
comparison_df['sector_returns'] = sector_returns_df.cumsum().iloc[-1]
comparison_df['portfolio_returns'] = comparison_df['portfolio_weights'] * comparison_df['stocks_returns']
comparison_df['benchmark_returns'] = comparison_df['benchmark_weights'] * comparison_df['sector_returns']

# The Alphas (Portfolio - Benchmark)
comparison_df['alphas'] = comparison_df['stocks_returns'] - comparison_df['sector_returns']
comparison_df['weights_diff'] = comparison_df['portfolio_weights'] - comparison_df['benchmark_weights']

comparison_df

In [142]:
# Check the sum
comparison_df.sum()

In [143]:
# Calculate the Brinson-Fachler Allocation Effect
allocation_effect = comparison_df['weights_diff'] * (comparison_df['sector_returns'] - comparison_df['benchmark_returns'].sum())
allocation_effect.name = 'allocation_effect'

allocation_effect

In [158]:
# Calculate the Brinson-Fachler Selection Effect
selection_effect = comparison_df['benchmark_weights'] * comparison_df['alphas']
selection_effect.name = 'selection_effect'

selection_effect

In [159]:
# Calculate the Brinson-Fachler Intersection Effect
interaction_effect = comparison_df['weights_diff'] * comparison_df['alphas']
interaction_effect.name = 'interaction_effect'

interaction_effect

In [160]:
# Calculate the Total Effect
bf_total_attribution = (
        allocation_effect + 
        selection_effect +
        interaction_effect
)

bf_total_attribution.name = 'bf_total_attribution'

bf_total_attribution

In [161]:
# Show
brinson_fachler_df = pd.DataFrame({
    'allocation': allocation_effect,
    'selection': selection_effect,
    'interaction': interaction_effect,
    'total': bf_total_attribution
})

brinson_fachler_df

In [162]:
brinson_fachler_df.sum().round(6)

In [163]:
# Divide the DataFrame by the Excess Return
norm_brinson_fachler_df = (brinson_fachler_df/brinson_fachler_df['total'].sum()) * 100

norm_brinson_fachler_df

In [164]:
norm_brinson_fachler_df.sum().round(2)

In [165]:
# Plotting individual attribution effects per asset
fig, ax = plt.subplots()

bar_width = 0.2
x = np.arange(len(norm_brinson_fachler_df))

# Plot each component
ax.bar(x - bar_width, norm_brinson_fachler_df["allocation"], width=bar_width, label="Allocation")
ax.bar(x, norm_brinson_fachler_df["selection"], width=bar_width, label="Selection")
ax.bar(x + bar_width, norm_brinson_fachler_df["interaction"], width=bar_width, label="Interaction")

# Formatting
ax.set_xticks(x)
ax.set_xticklabels(norm_brinson_fachler_df.index)
ax.set_ylabel("Attribution Effect (bps)")
ax.set_title("Brinson-Fachler Attribution by Asset")
ax.legend()
ax.axhline(0, color="gray", linewidth=0.8)

plt.tight_layout()
plt.show()

### Brinson-Hood-Beebower Model ###

To understand the differences between the BHB Model and the Brinson-Fachler Model, the reader can refer to Section 2 of this module’s PDF. There, you will find a detailed explanation of why the BHB Model is also used.

In [166]:
# Let us define the portfolios

# Quadrant IV Portfolio
pure_active_portfolio = stock_returns_df @ portfolio_weights
pure_active_portfolio.name = 'pure_active_portfolio_returns'

pure_active_portfolio 

In [167]:
# Quadrant I Portfolio
pure_pasive_portfolio = sector_returns_df @ norm_bench_weights
pure_pasive_portfolio.name = 'pure_pasive_portfolio_returns'

pure_pasive_portfolio

In [168]:
# Quadrant II Portfolio (Active Allocation Passive Selection)
aaps_portfolio = sector_returns_df @ portfolio_weights
aaps_portfolio.name = 'aaps_portfolio_returns'

aaps_portfolio

In [169]:
# Quadrant III Portfolio (Passive Allocation Active Selection)
paas_portfolio = stock_returns_df @ norm_bench_weights
paas_portfolio.name = 'paas_portfolio_returns'

paas_portfolio

In [170]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(pure_pasive_portfolio.cumsum(), label='Quadrant I Portfolio', alpha=1)
plt.plot(aaps_portfolio.cumsum(), label='Quadrant II Portfolio', alpha=1)
plt.plot(paas_portfolio.cumsum(), label='Quadrant III Portfolio', alpha=1)
plt.plot(pure_active_portfolio.cumsum(), label='Quadrant IV Portfolio', alpha=1)

# Config
plt.title('Portfolio Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show() 

In [172]:
# Now calculate the cumulative returns
quadrant_1_return = pure_pasive_portfolio.cumsum().iloc[-1]
quadrant_2_return = aaps_portfolio.cumsum().iloc[-1]
quadrant_3_return = paas_portfolio.cumsum().iloc[-1]
quadrant_4_return = pure_active_portfolio.cumsum().iloc[-1]

print(f'Quadrant 1 Return: {quadrant_1_return}')
print(f'Quadrant 2 Return: {quadrant_2_return}')
print(f'Quadrant 3 Return: {quadrant_3_return}')
print(f'Quadrant 4 Return: {quadrant_4_return}')

In [173]:
# The Excess Returns (we calculated it previously)
excess_returns

In [184]:
# Market Timing Effect:
market_timing_effect = quadrant_2_return - quadrant_1_return

print(f'Market Timing Effect: {market_timing_effect}')

In [185]:
# Security Selection Effect:
security_selection_effect = quadrant_3_return - quadrant_1_return

print(f'Security Selection Effect: {security_selection_effect}')

In [186]:
# Other Effects:
other_effects = quadrant_4_return + quadrant_1_return - quadrant_2_return - quadrant_3_return

print(f'Other Effects: {other_effects}')

In [187]:
# Total Effect (must be equal to the excess returns)
total_effect = market_timing_effect + security_selection_effect + other_effects

print(f'Total Effect: {total_effect}')

In [180]:
# Check that both models provide the same results
brinson_fachler_df.sum()